# Pokemon dataset

In [3]:

import pandas as pd
import numpy as np
import traceback as tb
import time

import plotly.graph_objects as go
from ipywidgets import interact
from PIL import Image, ImageFilter  # PIL is a popular image processing package "pillow"

![Sample images](https://lambdalabs.com/hs-fs/hubfs/2.%20Images/Images%20-%20Blog%20Posts/2022%20-%20Blog%20Images/image--3-.png?width=1152&height=768&name=image--3-.png)

In [1]:
from datasets import load_dataset
ds = load_dataset("lambdalabs/pokemon-blip-captions", split="train")

Found cached dataset parquet (C:/Users/north/.cache/huggingface/datasets/lambdalabs___parquet/lambdalabs--pokemon-blip-captions-baa94796864cc987/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [4]:
ds.column_names

['image', 'text']

In [16]:
ds

Dataset({
    features: ['image', 'text'],
    num_rows: 833
})

In [32]:
# below is slow updating
from plotly.subplots import make_subplots

#display(noisy_img)
pokemon_widget = go.FigureWidget()
pokemon_widget.add_trace(go.Image(z=np.array(ds[0]["image"])))

@interact(row_id=(0, 832, 1), continuous_update=False)
def show_sample(row_id=0):
    sample = ds[row_id] # each sample consists of an image and a text description
    img = sample["image"]
    with pokemon_widget.batch_update():
        pokemon_widget.data[0].z=np.array(img)
        pokemon_widget.update_xaxes(showticklabels=False)
        pokemon_widget.update_yaxes(showticklabels=False)
        pokemon_widget.layout.title = f"{row_id}: {sample['text']}"
        
display(pokemon_widget)

interactive(children=(IntSlider(value=0, description='row_id', max=832), Output()), _dom_classes=('widget-inte…

FigureWidget({
    'data': [{'type': 'image',
              'uid': '0fb4816e-68c7-491d-a5af-942cf522f390',
   …

In [14]:
type(ds)

datasets.arrow_dataset.Dataset

# Dataset type

Iteratable for image and text.


## a trivial example

In [4]:
from datasets.arrow_dataset import Dataset
N= 100 # number of data points
img_tensor = np.zeros((N, 512, 512, 3))
captions = []
for i in range(N):
    captions.append(f"hello: {i}")
    
captioned_imgs = {
    "image": img_tensor,
    "text": captions
}
#from datasets import Dataset
out = Dataset.from_dict(captioned_imgs)

out

Dataset({
    features: ['image', 'text'],
    num_rows: 100
})

## robot parts dataset

YOU are in charge to create this!


In [ ]:
# suppose we store 5 images in d:\dev\ai\SouthHighAI\data\VRCRobots folder and a single ImageText.txt there describes all images (1 image per line).
# The images are obtained by Google "VEX VRC Robot". Their sizes vary and no guarantee they are 512x512 or even square.


In [5]:
from datasets.arrow_dataset import Dataset
n = 5
vrc_folder = r"d:\dev\ai\SouthHighAI\data\VRCRobots\\"

with open(vrc_folder + "ImageText.txt", "r") as f:
    lines = f.readlines()
    
image_caption_dic = {}

for line in lines:
    toks = line.split(":")
    if len(toks) != 2:
        raise ValueError(f"ONLY TWO tokens allowed: {line}")
        
    i = int(toks[0])
    image_caption_dic[i] = toks[1]
    
vrc_img_tensor = np.zeros( (n, 512, 512, 3) )
vrc_captions = image_caption_dic.values()

for i in range(1, 6):
    img = Image.open(vrc_folder + str(i) + ".png")
    img = img.resize((512, 512), Image.LANCZOS).convert("RGB")
    vrc_img_tensor[i-1] = img
    

In [23]:

vrc_captioned_imgs = {"train": {
    "image": vrc_img_tensor,
    "text": vrc_captions
}
                     }
                     
#from datasets import Dataset
vrc_out = Dataset.from_dict(vrc_captioned_imgs)

vrc_out

Dataset({
    features: ['train'],
    num_rows: 2
})

In [20]:
# below is slow updating
from plotly.subplots import make_subplots

#display(noisy_img)
vrc_widget = go.FigureWidget()
vrc_widget.add_trace(go.Image(z=np.array(vrc_out[0]["image"])))

@interact(row_id=(0, 4, 1), continuous_update=False)
def show_vrc_sample(row_id=0):
    sample = vrc_out[row_id] # each sample consists of an image and a text description
    img = sample["image"]
    with pokemon_widget.batch_update():
        vrc_widget.data[0].z=np.array(img)
        vrc_widget.update_xaxes(showticklabels=False)
        vrc_widget.update_yaxes(showticklabels=False)
        vrc_widget.layout.title = f"{row_id}: {sample['text']}"
        
display(vrc_widget)

interactive(children=(IntSlider(value=0, description='row_id', max=4), Output()), _dom_classes=('widget-intera…

FigureWidget({
    'data': [{'type': 'image',
              'uid': '58f45222-7e5f-4089-b22d-bbd6fb6dc0c0',
   …

In [24]:
vrc_out.save_to_disk(vrc_folder + "vrc_robot_data52")

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]

In [7]:
vrc_in = Dataset.load_from_disk(vrc_folder + "vrc_robot_data")

In [8]:
vrc_in


Dataset({
    features: ['image', 'text'],
    num_rows: 5
})

In [16]:
from datasets import load_dataset
vex_in = load_dataset("imagefolder", data_dir=r"d:\dev\ai\SouthHighAI\data\VexRobots", split="train", )

Found cached dataset imagefolder (C:/Users/north/.cache/huggingface/datasets/imagefolder/default-848ee14aa76757c6/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


In [18]:
vex_in[0]["text"]

KeyError: 'text'

In [27]:
vex_in2 = Dataset.load_from_disk(r"d:\dev\ai\SouthHighAI\data\VexRobots\train")

FileNotFoundError: No such files: 'd:\dev\ai\SouthHighAI\data\VexRobots\train\dataset_info.json', nor 'd:\dev\ai\SouthHighAI\data\VexRobots\train\state.json' found. Expected to load a `Dataset` object but provided path is not a `Dataset`.